In [1]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [29]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [36]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 1.4 s, sys: 6.79 ms, total: 1.41 s
Wall time: 1.53 s


In [41]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 863 ms, sys: 9.71 ms, total: 873 ms
Wall time: 875 ms


In [45]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 335 ms, sys: 6.79 ms, total: 342 ms
Wall time: 339 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

In [44]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 3.2 ms, sys: 2.78 ms, total: 5.98 ms
Wall time: 6.9 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

# Practice time

In [ ]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [47]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as 
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 1371  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         6317  -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         6763  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         9838  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         16403 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 1371       DAY  OTHERS          THEFT/OTHER   
         6317       DAY  OTHERS         THEFT F/AUTO   
         6763       DAY  OTHERS         THEFT F/AUTO   
         9838   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16403      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 1371        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         6317        1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         6763   2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         9838        1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         16403    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 1371   ...        5801.0    Precinct 129  38.899121 -77.021926   
         6317   ...        5303.0     Precinct 15  38.909638 -77.040089   
         6763   ...        4100.0     Precinct 13  38.913346 -77.050526   
         9838   ...       10700.0     Precinct 17  38.905648 -77.039822   
         16403  ...        5302.0     Precinct 15  38.913720 -77.038489   

                            BID              START_DATE  \
20123250 1371          DOWNTOWN  2020/08/29 16:05:40+00   
         6317               NaN  2020/08/28 22:00:23+00   
         6763               NaN  2020/08/26 22:00:29+00   
         9838   GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         16403              NaN  2020/08/28 20:55:00+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 1371   2020/08/29 16:08:33+00  398247584             NaN   
         6317   2020/08/29 08:00:27+00  398455909             NaN   
         6763   2020/08/27 12:00:51+00  398456841             NaN   
         9838   2020/08/29 19:00:05+00  398514253             NaN   
         16403  2020/08/28 21:05:00+00  398535856             NaN   

                               report_dt  
20123250 1371  2020-08-29 17:15:19+00:00  
         6317  2020-08-29 16:05:18+00:00  
         6763  2020-08-29 16:45:57+00:00  
         9838  2020-08-29 22:04:46+00:00  
         16403 2020-08-29 14:29:59+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [48]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 3817  -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   
         3819  -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
         3820  -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         8620  -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14715 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   

                  SHIFT  METHOD              OFFENSE  \
20165648 3817       DAY  OTHERS         THEFT F/AUTO   
         3819       DAY  OTHERS         THEFT F/AUTO   
         3820   EVENING  OTHERS          THEFT/OTHER   
         8620       DAY  OTHERS          THEFT/OTHER   
         14715  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                     BLOCK    XBLOCK  \
20165648 3817          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         3819               300 - 399 BLOCK OF G STREET NE  399886.0   
         3820   300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         8620    600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14715          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 3817   137456.0  ...       10602.0     Precinct 83  38.904961   
         3819   136784.0  ...        8301.0     Precinct 83  38.898907   
         3820   136899.0  ...        5900.0    Precinct 143  38.899942   
         8620   135255.0  ...        6500.0     Precinct 89  38.885133   
         14715  137478.0  ...       10603.0    Precinct 144  38.905159   

                LONGITUDE           BID              START_DATE  \
20165648 3817  -76.997314           NaN  2020/11/19 23:45:48+00   
         3819  -77.001314           NaN  2020/11/20 15:30:02+00   
         3820  -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         8620  -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   
         14715 -77.005891          NOMA  2020/11/20 20:15:26+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 3817   2020/11/20 03:00:00+00  398253755             NaN   
         3819   2020/11/20 18:25:35+00  398253757             NaN   
         3820   2020/11/20 22:08:28+00  398253758             NaN   
         8620                      NaN  398487694             NaN   
         14715  2020/11/20 21:46:24+00  398533341             NaN   

                               report_dt  
20165648 3817  2020-11-20 14:45:06+00:00  
         3819  2020-11-20 18:56:18+00:00  
         3820  2020-11-20 22:07:10+00:00  
         8620  2020-11-20 12:46:32+00:00  
         14715 2020-11-20 22:17:27+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [ ]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here

### 2.2 apply it to one of the focal crimes

### 2.3 Use apply to cover all the other focal crimes